# Pandas cheatsheet

- toc: true 
- hide: false
- branch: master
- search_exclude: false
- badges: true
- comments: true
- categories: [numpy, pandas]

Good resources:
 - [Real python](https://realpython.com/python-data-cleaning-numpy-pandas/?s=09)
 - [Pandas Docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

In [1]:
# hide
import pandas as pd
import numpy as np

## Re-order columns

In [ ]:
df = df.reindex(columns=FIRST_COLS + sorted(list(REMAINING_COLS)))

## Assign: Create new columns easily from existing columns

In [ ]:
# Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html
df.assign(
    temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,
    temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9
)

## Tall to Wide form: `pd.pivot`

Suppose we have a dataset of student grades with each student having multiple rows for each school subject grade. We want each row to correspond to a single student entry: all grades for all subjects for the students should be in a single row of the dataframe. 

In [ ]:
df_wide = df.pivot_table(values='Grade', # Score in each subject
                         index=['ID', 'School'], # Student ID and school
                         columns='Subjects', # Name of the subject
                         aggfunc='last') 

## Wide to tall: `pd.melt`

Never used it but seems the opposite of `pd.pivot_table` above

https://pandas.pydata.org/docs/reference/api/pandas.melt.html

In [ ]:
pd.melt(df, id_vars=['A'], value_vars=['B', 'C'])

## Create a binary column based on two separate columns

In [ ]:
df.loc[df['X'].notnull(), 'X_true'] = 1
df.loc[df['X'].isnull(),  'X_true'] = 0

## One Hot Encoding: custom name for the columns

Create the columns based on OHE of a given column: 

```python
['cluster 0', 'cluster 1', 'cluster 2']
```

**Bad** way

In [ ]:
# What I have been using since the beginning of time
# df_clusters_save.loc[df_clusters_save['clusters_kmeans']==0, 'cluster 0'] = 1
# df_clusters_save.loc[df_clusters_save['clusters_kmeans']!=0, 'cluster 0'] = 0
# df_clusters_save.loc[df_clusters_save['clusters_kmeans']==1, 'cluster 1'] = 1
# df_clusters_save.loc[df_clusters_save['clusters_kmeans']!=1, 'cluster 1'] = 0
# df_clusters_save.loc[df_clusters_save['clusters_kmeans']==2, 'cluster 2'] = 1
# df_clusters_save.loc[df_clusters_save['clusters_kmeans']!=2, 'cluster 2'] = 0
# df_clusters_save.columns

**Good** way

In [ ]:
dummies = pd.get_dummies(df_clusters_save['clusters_kmeans']).rename(columns=lambda x: 'cluster ' + str(x))
df_clusters_save = pd.concat([df_clusters_save, dummies], axis=1)
df_clusters_save.columns

## Duplicates based on certain columns

In [ ]:
duplicates = df[df.duplicated(subset=['ID', 'School'], keep=False)]
print(f"Duplicates found: {duplicates.shape[0]}")

# drop duplicates
df_students.drop_duplicates(subset=['ID', 'School'], keep='last', inplace=True)

## Force numeric dtype

In [ ]:
for col in df.columns.tolist()[1:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')

## Swedish/other language characters

8: 8-bit encoding

sig: signature; apparently adds signature at the beginning of the file so that software like microsoft excel / power bi can read it correctly

In [ ]:
df.to_csv('names_in_swedish_german_other_languages.csv', index=False, encoding='utf-8-sig')

## Row entries of a column that belong to a list

For column `Enhet`, include/exclude only the entries that belong to a pre-defined list.

In [1]:
# Exclude entries
df[~df['Enhet'].isin(EXCLUDE_SCHOOLS)]
# Include entries
df[df['Enhet'].isin(INCLUDE_SCHOOLS)]

## `dropna` based on a column

In [ ]:
df.dropna(subset=['School'], inplace=True)

## Rename columns

In [ ]:
COLUMN_NAMES = {
    'X': 'Velocity', 
    'Y': 'Position',
    'Z': 'Time'
}

df.rename(columns=COLUMN_NAMES, inplace=True)

# caveat: does NOT always work!

## Replace row values

Has a lot of issues, so better to re-assign instead of using `inplace=True`.

In [ ]:
SCHOOL_NAMES = {
    "Jan": "January", 
    "Jan.": "January"
}

# inplace does not always work
# df.replace(SCHOOL_NAMES, inplace=True)

# Better
df = df.replace(SCHOOL_NAMES)

## String processing

In [ ]:
# Convert to string and split based on '-'
df_schools_list = df.school.str.split('-').tolist()
# Get rid of white space
school_names = [long_name[-1].strip() for long_name in df_schools_list]
# Rename schools
df.school = school_names